<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PreferredAI/tutorials/blob/master/multimodal-www23/03_explainability.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PreferredAI/tutorials/blob/master/multimodal-www23/03_explainability.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Explainable Recommendations


While the main objective of a recommender system is to identify the items to be recommended to a user, providing explanations to accompany the recommendations would be more persuasive as well as engender trust and transparency.  There are different types of explanations.  In this tutorial, we explore explainable recommendation approaches that rely on user product aspect-level sentiment for modeling explanations.

## 1. Setup

In [ ]:
!pip install --quiet cornac==1.15.0

In [2]:
import os
import sys
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import cornac
from cornac.utils import cache
from cornac.datasets import amazon_toy
from cornac.eval_methods import RatioSplit
from cornac.data import Reader, SentimentModality
from cornac.models import EFM, MTER, NMF, BPR

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

SEED = 42
VERBOSE = True

System version: 3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
Cornac version: 1.15.0


## 2. Aspect-Level Sentiments
To model fine-grained product aspect-ratings. Several works rely on sentiment analysis to extract aspect sentiment from product reviews. In other words, each review is now a list of aspect sentiments. Along with product rating, we also have aspect sentiments expressed in users' reviews. Here, we work with Toys and Games dataset, a sub-category of [Amazon reviews](http://jmcauley.ucsd.edu/data/amazon/).

Below are some examples of aspect-level sentiments that have been extracted from users' reviews of items.

In [3]:
sentiment_fpath = cache(url='https://github.com/PreferredAI/static-data/raw/main/cornac/datasets/amazon_toy/sentiment.zip',
                        unzip=True, relative_path='amazon_toy/sentiment.txt')
sentiment = Reader().read(sentiment_fpath, fmt='UITup', sep=',', tup_sep=':')
samples = sentiment[:10]
pd.DataFrame.from_dict({
  "user": [tup[0] for tup in samples],
  "item": [tup[1] for tup in samples],
  "aspect-level sentiment": [tup[2] for tup in samples]
})

Data from https://github.com/PreferredAI/static-data/raw/main/cornac/datasets/amazon_toy/sentiment.zip
will be cached into /root/.cornac/amazon_toy/sentiment.txt


0.00B [00:00, ?B/s]

Unzipping ...
File cached!


,user,item,aspect-level sentiment
0,A012468118FTQAINEI0OQ,B00005BZM6,"[(paint, great, 1)]"
1,A012468118FTQAINEI0OQ,B001HA9JOA,"[(game, great, 1), (money, worth, 1)]"
2,A012468118FTQAINEI0OQ,B002BY2BVE,"[(paint, fun, 1), (item, well, 0)]"
3,A012468118FTQAINEI0OQ,B007U7M0LI,"[(price, sturdy, 1)]"
4,A012468118FTQAINEI0OQ,B00804BCO6,"[(gift, great, 1)]"
5,A0182108CPDLPRCXQUZQ,B002IUNLLK,"[(toy, best, 1), (heavy, cool, 1)]"
6,A0182108CPDLPRCXQUZQ,B007WYU7R8,"[(toy, great, 1)]"
7,A0182108CPDLPRCXQUZQ,B00ABY8WVO,"[(toy, love, 1)]"
8,A0182108CPDLPRCXQUZQ,B00AFP86KG,"[(toy, love, 1)]"
9,A0182108CPDLPRCXQUZQ,B00BJT861Q,"[(figure, well, 1), (toy, well, 1)]"


In [4]:
# Load rating and sentiment information
rating_fpath = cache(url='https://github.com/PreferredAI/static-data/raw/main/cornac/datasets/amazon_toy/rating.zip',
                     unzip=True, relative_path='amazon_toy/rating.txt')
rating = Reader(min_item_freq=20).read(rating_fpath, fmt='UIRT', sep=',')

# Use Sentiment Modality for aspect-level sentiment data
sentiment_modality = SentimentModality(data=sentiment)

rs = RatioSplit(
    data=rating,
    test_size=0.2,
    exclude_unknowns=True,
    sentiment=sentiment_modality,
    verbose=VERBOSE,
    seed=SEED,
)
print("Total number of aspects:", rs.sentiment.num_aspects)
print("Total number of opinions:", rs.sentiment.num_opinions)

id_aspect_map = {v:k for k, v in rs.sentiment.aspect_id_map.items()}
id_opinion_map = {v:k for k, v in rs.sentiment.opinion_id_map.items()}

Data from https://github.com/PreferredAI/static-data/raw/main/cornac/datasets/amazon_toy/rating.zip
will be cached into /root/.cornac/amazon_toy/rating.txt


0.00B [00:00, ?B/s]

Unzipping ...
File cached!
Total number of aspects: 429
Total number of opinions: 2604


## 3. Explicit Factor Model (EFM)

EFM model extends Non-negative Matrix Factorization (NMF) with the additional information from **aspect-level sentiments**.  The objective is to learn user, item, and aspect latent factors to explain user-item ratings, users' interest in certain aspects of the items, as well as the quality of items according to those aspects.  In a nutshell, EFM factorizes three matrices: *rating matrix*, *user-aspect attention matrix*, and *item-aspect quality matrix*.  Let's take a look at what the later two matrices are.

In [5]:
efm = EFM()
efm.train_set = rs.train_set
_, X, Y = efm._build_matrices(rs.train_set)

<ipython-input-5-bd94163efa07>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _, X, Y = efm._build_matrices(rs.train_set)
<ipython-input-5-bd94163efa07>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _, X, Y = efm._bui

### User-Aspect Attention Matrix

Let $\mathcal{F} = \{f_1, f_2, \dots, f_F\}$ be the set of aspects (e.g., screen, earphone). 

Let $\mathbf{X} \in \mathbb{R}^{N \times F}$ be a sparse aspect matrix for $N$ users and $F$ aspects, whereby each element $x_{if} \in \mathbf{X}$  indicates the degree of **attention** by user $i$ on aspect $f$, defined as follows:

\begin{equation}
x_{if} = \
\begin{cases}
0, & \text{if user $i$ never mentions aspect $f$} \\
1 + (N-1)\left(\frac{2}{1+\exp(-t_{if})}-1\right), & \text{otherwise}
\end{cases}
\end{equation}

where $N=5$ is the highest rating score, $t_{if}$ is the frequency of user $i$ mentions aspect $f$ across all her reviews.

For illustration purpose, we show a small matrix $\mathbf{X}$ of 5 users and 5 aspects below.

In [6]:
n_users = 5
n_aspects = 5
pd.DataFrame(
  data=X[:n_users, :n_aspects].A,
  index=[f"User {u + 1}" for u in np.arange(n_users)],
  columns=[f"{id_aspect_map[i]}" for i in np.arange(n_aspects)]
)

,paint,game,money,item,price
User 1,0.0,0.000000,0.0,0.000000,2.848469
User 2,0.0,0.000000,0.0,0.000000,0.000000
User 3,0.0,4.620593,0.0,2.848469,4.620593
User 4,0.0,4.856110,0.0,0.000000,0.000000
User 5,0.0,0.000000,0.0,0.000000,0.000000


In the example below, we can see that *User 4* finds the aspect *game* important, whereas *User 3* is concerned with *game* as well as *price*.

### Item-Aspect Quality Matrix


Let $\mathbf{Y} \in \mathbb{R}^{M \times F}$ be a sparse aspect matrix for $M$ items and $F$ aspects, whereby $y_{jf} \in \mathbf{Y}$ indicates the **quality** of item $j$ on aspect $f$, defined as follows:

\begin{equation}
y_{jf} = \
\begin{cases}
0, & \text{if item $j$ was never reviewed on aspect $f$} \\
1 + (N - 1) \left( \frac{1}{1+\exp(-s_{jf})} \right), & \text{otherwise}
\end{cases}
\end{equation}

where $s_{jf}$ is the sum of sentiment values with which item $j$ has been mentioned with regards to aspect $f$ across all its reviews.

We show a small matrix $Y$ of 5 items and 5 aspects below:

In [7]:
n_items = 5
n_aspects = 5
pd.DataFrame(
  data=Y[:n_items, :n_aspects].A,
  index=[f"Item {u + 1}" for u in np.arange(n_items)],
  columns=[f"{id_aspect_map[i]}" for i in np.arange(n_aspects)]
)

,paint,game,money,item,price
Item 1,3.924234,0.000000,3.000000,0.000000,3.924234
Item 2,0.000000,0.000000,0.000000,4.523188,4.523188
Item 3,0.000000,4.523188,0.000000,0.000000,0.000000
Item 4,0.000000,0.000000,0.000000,0.000000,3.924234
Item 5,3.924234,0.000000,3.924234,4.523188,4.523188


We see from the example above that *Item 3* has a positive quality in the aspect *game*, whereas *Item 5* has positive quality on the other 4 aspects.

### Optimization

As these matrices are sparse, for prediction, EFM jointly factorizes $X$ and $Y$ matrices along with rating matrix $R$.   Learning the latent factors can be done via minimizing the following loss function:

\begin{align}
&\mathcal{L}(\mathbf{U_1, U_2, V, H_1, H_2} | \lambda_x, \lambda_y, \lambda_u, \lambda_h, \lambda_v) = ||\mathbf{U_1} \mathbf{U_2}^T + \mathbf{H_1} \mathbf{H_2}^T - \mathbf{R}||_F^2 + \lambda_x ||\mathbf{U_1} \mathbf{V}^T - \mathbf{X}||_F^2 + \lambda_y ||\mathbf{U_2} \mathbf{V}^T - \mathbf{Y}||_F^2 + \lambda_u(||\mathbf{U_1}||_F^2+||\mathbf{U_2}||_F^2) + \lambda_h(||\mathbf{H_1}||_F^2+||\mathbf{H_2}||_F^2) + \lambda_v ||\mathbf{V}||_F^2 \\
&\text{such that: }  \forall_{i, k} u_{ik} \ge 0, \forall_{j, k} v_{jk} \ge 0
\end{align}

The can be solved as a constrained optimization problem. 


Let's conduct an experiment with EFM model and compare with NMF as a baseline.

In [8]:
efm = EFM(
  num_explicit_factors=40,
  num_latent_factors=60,
  num_most_cared_aspects=15,
  rating_scale=5.0,
  alpha=0.85,
  lambda_x=1,
  lambda_y=1,
  lambda_u=0.01,
  lambda_h=0.01,
  lambda_v=0.01,
  max_iter=100,
  verbose=VERBOSE,
  seed=SEED,
)

# compare to baseline NMF
nmf = NMF(k=100, max_iter=100, verbose=VERBOSE, seed=SEED)

eval_metrics = [
  cornac.metrics.RMSE(),
  cornac.metrics.NDCG(k=50),
  cornac.metrics.AUC()
]

cornac.Experiment(
  eval_method=rs, models=[nmf, efm], metrics=eval_metrics
).run()


TEST:
...
    |   RMSE |    AUC | NDCG@50 | Train (s) | Test (s)
--- + ------ + ------ + ------- + --------- + --------
NMF | 0.8027 | 0.5418 |  0.0093 |    4.4047 |  10.8052
EFM | 0.7315 | 0.5536 |  0.0105 |   10.8713 |  13.9718



### Refining Ranking Prediction

With EFM model, you can refine the recommendation after training by experimenting with different values of: 
*   `num_most_cared_aspects` ($k$): integer, value range $\in[0, 429]$ as we have $429$ aspects in total
*   `alpha` $\in [0,1]$

These parameters will affect ranking performance of the EFM model, as the ranking score is predicted as follow:

$$
ranking\_score = \alpha \cdot \frac{\sum_{f \in C_i}{\hat{x}_{if}\cdot\hat{y}_{jf}}}{k \cdot N} + (1-\alpha)\cdot\hat{r}_{ij}
$$

In [9]:
alpha = 0.9 # alpha value in range [0,1]
num_most_cared_aspects = 100

eval_metrics = [
  cornac.metrics.NDCG(k=50),
  cornac.metrics.AUC()
]

cornac.Experiment(
  eval_method=rs,
  models=[
    EFM(
      alpha=alpha,
      num_most_cared_aspects=num_most_cared_aspects,
      init_params={'U1': efm.U1, 'U2': efm.U2, 'H1': efm.H1, 'H2': efm.H2, 'V': efm.V},
      trainable=False,
      verbose=VERBOSE,
      seed=SEED
    )
  ],
  metrics=eval_metrics
).run()


TEST:
...
    |    AUC | NDCG@50 | Train (s) | Test (s)
--- + ------ + ------- + --------- + --------
EFM | 0.5549 |  0.0107 |    0.0006 |  17.8007



### Recommendation Explanation with EFM

Given a user and an item, EFM model is able of predicting **user's attention scores** as well as **item's quality scores** regarding the aspects.  Those scores with the corresponding aspects will be the explanation on why a user *likes* or *dislikes* an item.

Let's take a look at an example below. Feel free to explore other users and items!

In [10]:
UIDX = 1
IIDX = 4
num_top_cared_aspects = 10

id_aspect_map = {v:k for k, v in rs.sentiment.aspect_id_map.items()}

predicted_user_aspect_scores = np.dot(efm.U1[UIDX], efm.V.T)
predicted_item_aspect_scores = np.dot(efm.U2[IIDX], efm.V.T)

top_cared_aspect_ids = (-predicted_user_aspect_scores).argsort()[:num_top_cared_aspects]
top_cared_aspects = [id_aspect_map[aid] for aid in top_cared_aspect_ids]
pd.DataFrame.from_dict({
  "aspect": top_cared_aspects,
  "user_aspect_attention_score": predicted_user_aspect_scores[top_cared_aspect_ids],
  "item_aspect_quality_score": predicted_item_aspect_scores[top_cared_aspect_ids]
})


,aspect,user_aspect_attention_score,item_aspect_quality_score
0,toy,4.061583,4.679121
1,pieces,3.610667,3.844794
2,game,3.571417,4.211303
3,furby,3.571003,4.787230
4,doll,3.480350,4.326975
5,quality,3.468947,3.952712
6,really,3.368340,4.586976
7,gift,3.364289,4.410736
8,also,3.360865,4.076250
9,puzzle,3.294265,4.726531


EFM takes an aspect with the **highest score** in `item_aspect_quality_score` as the well-performing aspect, and an aspect with the **lowest score** in `item_aspect_quality_score` as the poorly-performing aspect. See example explanations in their templates below.

In [11]:
perform_well_aspect = top_cared_aspects[predicted_item_aspect_scores[top_cared_aspect_ids].argmax()]
perform_poorly_aspect = top_cared_aspects[predicted_item_aspect_scores[top_cared_aspect_ids].argmin()]

explanation = \
f"You might interested in [{perform_well_aspect}], on which this product perform well. \n\
You might interested in [{perform_poorly_aspect}], on which this product perform poorly."
print("EFM explanation:")
print(explanation)

EFM explanation:
You might interested in [furby], on which this product perform well. 
You might interested in [pieces], on which this product perform poorly.


## 4. Multi-Task Explainable Recommendation (MTER)

MTER model extends the concept of exploiting information from *Aspect-Level Sentiments* with tensor factorization (using Tucker Decomposition).   The model takes in the input of three tensors.  Let's go through each of them and see how they are constructed.



### Tensor\#1: User by Item by Aspect ($\mathbf{X}$)

Let $\mathbf{R} \in \mathbb{R}^{N \times M}$ be a sparse rating matrix of $N$ users and $M$ items.

Let $\mathbf{X} \in \mathbb{R}_{+}^{N \times M \times F}$ be a 3-dimensional tensor, each element $x_{ijf}$ indicates a relationship between user $i$, item $j$, and aspect $f$:

\begin{equation}
x_{ijf} = \
\begin{cases}
0, & \text{if aspect $f$ has not been mentioned by user $i$ about item $j$} \\
1 + (N-1)\left(\frac{1}{1+\exp(-s_{ijf})}\right), & \text{otherwise}
\end{cases}
\end{equation}

where $s_{ijf}$ is the sum of sentiment values with which item $j$ has been mentioned by user $i$ with regards to aspect $f$.

We can extend $\mathbf{X}$ into $\mathbf{\tilde{X}}$ with the rating matrix $\mathbf{R}$ as the last slice or the $(F + 1)^{\mathrm{th}}$ aspect (i.e., $\tilde{x}_{ij(F+1)} = r_{ij}$).

### Tensor\#2: User by Aspect by Opinion ($\mathbf{Y}^{U}$)

Let $\mathbf{Y}^{U} \in \mathbb{R}_{+}^{N \times F \times O}$ be a 3-dimensional tensor, each element $y^U_{ifo}$ indicates a relationship between user $i$, aspect $f$, and opinion $o$:

\begin{equation}
y^U_{ifo} = \
\begin{cases}
0, & \text{if user $i$ has not been used opinion $o$ to describe aspect $f$ positively} \\
1 + (N-1)\left(\frac{1}{1+\exp(-t_{ifo})}\right), & \text{otherwise}
\end{cases}
\end{equation}

where $t_{ifo}$ is the frequency with which user $i$ employs opinion $o$ to describe aspect $f$ positively across all her reviews.


### Tensor\#3: Item by Aspect by Opinion ($\mathbf{Y}^{I}$)

Let $\mathbf{Y}^{I} \in \mathbb{R}_{+}^{M \times F \times O}$ be a 3-dimensional tensor, each element $y^I_{jfo}$ indicates a relationship between item $j$, aspect $f$, and opinion $o$:

\begin{equation}
y^I_{jfo} = \
\begin{cases}
0, & \text{if item $j$ has not been described positively with opinion $o$ on aspect $f$} \\
1 + (N-1)\left(\frac{1}{1+\exp(-t_{jfo})}\right), & \text{otherwise}
\end{cases}
\end{equation}

where $t_{jfo}$ is the frequency with which item $j$ has been described positively with opinion $o$ on aspect $f$ positively across all its reviews.

### Optimization

MTER employs Tucker Decomposition to jointly factorize three tensors $\mathbf{\tilde{X}}$, $\mathbf{Y}^U$, and $\mathbf{Y}^I$.   In addition, MTER also optimizes for a ranking objective akin to BPR where:
*  Positive triples $\mathbf{T} = \{ j >_{i} j' | x_{ij(F+1)} \in \mathbf{R}^+ \land x_{ij'(F+1)} \in \mathbf{R}^- \}$
*  For aspect (F + 1), which is the overall rating, user $i$ prefers item $j$ to item $j'$

Learning the latent factors can be done via minimizing the following loss function:

\begin{align}
&\mathcal{L}(\mathbf{U, V, Z, W, C_1, C_2, C_3} | \lambda_B, \lambda) = ||\mathbf{\tilde{X}} - \mathbf{\hat{X}}||_F^2 + ||\mathbf{Y}^U - \hat{\mathbf{Y}}^U||_F^2 + ||\mathbf{Y}^I - \hat{\mathbf{Y}}^I||_F^2 - \lambda_B \sum_{j >_i j'} \ln(1 + \exp{(-(\hat{x}_{ij(F+1)} - \hat{x}_{ij'(F+1)}))}) + \lambda(||\mathbf{U}||_F^2+||\mathbf{V}||_F^2+||\mathbf{Z}||_F^2+||\mathbf{W}||_F^2 +||\mathbf{C_1}||_F^2 +||\mathbf{C_2}||_F^2 +||\mathbf{C_3}||_F^2) \\
&\text{such that: }  \mathbf{U} \ge 0, \mathbf{V} \ge 0, \mathbf{Z} \ge 0, \mathbf{W} \ge 0, \mathbf{C_1} \ge 0, \mathbf{C_2} \ge 0, \mathbf{C_3} \ge 0
\end{align}


The can be solved as a constrained optimization problem. 


Let's conduct an experiment with MTER model and compare with the BPR baseline.

In [12]:
mter = MTER(
  n_user_factors=10,
  n_item_factors=10,
  n_aspect_factors=10,
  n_opinion_factors=10,
  n_bpr_samples=1000,
  n_element_samples=50,
  lambda_reg=0.1,
  lambda_bpr=10,
  max_iter=3000,
  lr=0.5,
  verbose=VERBOSE,
  seed=SEED,
)

# compare to baseline BPR
bpr = BPR(k=10, verbose=VERBOSE, seed=SEED)

eval_metrics = [
  cornac.metrics.NDCG(k=50),
  cornac.metrics.AUC()
]

# Instantiate and run an experiment
cornac.Experiment(
  eval_method=rs, models=[bpr, mter], metrics=eval_metrics,
).run()


TEST:
...
     |    AUC | NDCG@50 | Train (s) | Test (s)
---- + ------ + ------- + --------- + --------
BPR  | 0.6271 |  0.0314 |    1.1174 |   7.5330
MTER | 0.7185 |  0.0357 |   47.8754 |   6.6390



### Recommendation Explanation with MTER

* To provide recommendation to user $i$, we rank items $j$ in terms of the predicted rating scores: $\hat{x}_{ij(F+1)}$

* To determine which aspect $f$ of product $j$ a user $i$ cares about, we rank aspects $f$ in terms of: $\hat{x}_{ijf}$

* To determine which opinion phrases $o$ to use when describing aspect $f$ while recommending item $j$ to user $i$, we rank phrases in terms of: $\hat{y}^U_{ifo} \times \hat{y}^I_{jfo}$

Let's explore an example below on how we can generate explanations for recommendation by MTER model.

In [13]:
UIDX = 10
IIDX = 10
num_top_aspects = 2
num_top_opinions = 3

item_aspect_ids = np.array(list(set([
    tup[0]
    for idx in rs.sentiment.item_sentiment[IIDX].values()
    for tup in rs.sentiment.sentiment[idx]
])))

item_opinion_ids = np.array(list(set([
  tup[1]
  for idx in rs.sentiment.item_sentiment[IIDX].values()
  for tup in rs.sentiment.sentiment[idx]
])))

item_aspects = [id_aspect_map[idx] for idx in item_aspect_ids]

ts1 = np.einsum("abc,a->bc", mter.G1, mter.U[UIDX])
ts2 = np.einsum("bc,b->c", ts1, mter.I[IIDX])
predicted_aspect_scores = np.einsum("c,Mc->M", ts2, mter.A)

top_aspect_ids = item_aspect_ids[(-predicted_aspect_scores[item_aspect_ids]).argsort()[:num_top_aspects]]
top_aspects = [id_aspect_map[idx] for idx in top_aspect_ids]

top_aspect_opinions = []
mter_explanations = []
for top_aspect_id, top_aspect in zip(top_aspect_ids, top_aspects):
  ts1_G2 = np.einsum("abc,a->bc", mter.G2, mter.U[UIDX])
  ts2_G2 = np.einsum("bc,b->c", ts1_G2, mter.A[top_aspect_id])
  predicted_user_aspect_opinion_scores = np.einsum("c,Mc->M", ts2_G2, mter.O)

  ts1_G3 = np.einsum("abc,a->bc", mter.G3, mter.I[IIDX])
  ts2_G3 = np.einsum("bc,b->c", ts1_G3, mter.A[top_aspect_id])
  predicted_item_aspect_opinion_scores = np.einsum("c,Mc->M", ts2_G3, mter.O)

  predicted_aspect_opinion_scores = np.multiply(predicted_user_aspect_opinion_scores, predicted_item_aspect_opinion_scores)
  top_opinion_ids = item_opinion_ids[(-predicted_aspect_opinion_scores[item_opinion_ids]).argsort()[:num_top_opinions]]
  top_opinions = [id_opinion_map[idx] for idx in top_opinion_ids]
  top_aspect_opinions.append(top_opinions)

  # Generate explanation for top-1 aspect
  mter_explanations.append(f"Its {top_aspect} is [{'] ['.join(top_opinions)}].")

pd.DataFrame.from_dict({"aspect": top_aspects, "top_opinions": top_aspect_opinions, "explanation": mter_explanations})

,aspect,top_opinions,explanation
0,really,"[disappointed, great, like]",Its really is [disappointed] [great] [like].
1,addition,"[disappointed, great, fun]",Its addition is [disappointed] [great] [fun].


## References

1.   Zhang, Y., Lai, G., Zhang, M., Zhang, Y., Liu, Y., & Ma, S. (2014). Explicit factor models for explainable recommendation based on phrase-level sentiment analysis. In SIGIR (pp. 83-92).
2. Wang, N., Wang, H., Jia, Y., & Yin, Y. (2018). Explainable recommendation via multi-task learning in opinionated text data. In SIGIR (pp. 165-174). 
4. Salah, A., Truong, Q. T., & Lauw, H. W. (2020). Cornac: A Comparative Framework for Multimodal Recommender Systems. J. Mach. Learn. Res., 21, 95-1. https://cornac.preferred.ai